In [17]:
import os
import shutil


# data_dir = './videos/MRI515_T2/'
# video_dir = os.path.join(data_dir, 'img_in_jpg_T2_DINO_order')
# video_targ_dir = os.path.join(data_dir, 'img_in_jpg_L4L5_renamed')

# video_dir = os.path.join(data_dir, 'label_in_png_T2_DINO_order')
# video_targ_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')

# data_dir = './videos/MRI515_T1/'
# video_dir = os.path.join(data_dir, 'img_in_jpg_T1_DINO_order')
# video_targ_dir = os.path.join(data_dir, 'img_in_jpg_L4L5_renamed')
# video_dir = os.path.join(data_dir, 'label_in_png_T1_DINO_order')
# video_targ_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')

# data_dir = 'videos/AFL_MRI/raw_data'
# data_dir = 'videos/bed_rest_MRI/data_in_nii'
data_dir = 'videos/CT_tt'

video_dir = os.path.join(data_dir, 'img_in_jpg_2class_DINO_order')
video_targ_dir = os.path.join(data_dir, 'img_in_jpg_L4L5_renamed')

video_dir = os.path.join(data_dir, 'label_in_png_2class_DINO_order')
video_targ_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')




# # 确保目标文件夹存在
os.makedirs(video_targ_dir, exist_ok=True)

# 获取文件夹中的所有文件名
file_names = [f for f in os.listdir(video_dir) if f.endswith(".jpg") or f.endswith(".png")]

# 按排序结果复制文件并重命名
for idx, file_name in enumerate(sorted(file_names)):
    # 原文件路径
    old_path = os.path.join(video_dir, file_name)
    
    # 确定文件扩展名
    file_ext = os.path.splitext(file_name)[1]

    # 新文件名
    new_name = f"{idx:05d}{file_ext}"
    new_path = os.path.join(video_targ_dir, new_name)
    
    # 复制并重命名文件
    shutil.copy2(old_path, new_path)
    print(f"Copied: {old_path} -> {new_path}")

print("File copy and renaming completed!")


Copied: videos/CT_tt/label_in_png_2class_DINO_order/00000_s1355_label_L4L5_slice134.png -> videos/CT_tt/label_in_png_L4L5_renamed/00000.png
Copied: videos/CT_tt/label_in_png_2class_DINO_order/00001_s0476_label_L4L5_slice151.png -> videos/CT_tt/label_in_png_L4L5_renamed/00001.png
Copied: videos/CT_tt/label_in_png_2class_DINO_order/00002_s0034_label_L4L5_slice172.png -> videos/CT_tt/label_in_png_L4L5_renamed/00002.png
Copied: videos/CT_tt/label_in_png_2class_DINO_order/00003_s0863_label_L4L5_slice158.png -> videos/CT_tt/label_in_png_L4L5_renamed/00003.png
Copied: videos/CT_tt/label_in_png_2class_DINO_order/00004_s0636_label_L4L5_slice139.png -> videos/CT_tt/label_in_png_L4L5_renamed/00004.png
Copied: videos/CT_tt/label_in_png_2class_DINO_order/00005_s1377_label_L4L5_slice148.png -> videos/CT_tt/label_in_png_L4L5_renamed/00005.png
Copied: videos/CT_tt/label_in_png_2class_DINO_order/00006_s0472_label_L4L5_slice139.png -> videos/CT_tt/label_in_png_L4L5_renamed/00006.png
Copied: videos/CT_tt

In [18]:
import os
from PIL import Image
import numpy as np

# 设置数据目录
video_dirs = [
   os.path.join(data_dir, 'img_in_jpg_L4L5_renamed'),
   os.path.join(data_dir, 'label_in_png_L4L5_renamed')
]

# 根据目录设置目标大小
targ_size = 384
if 'TUM' in data_dir:
    targ_size = 400
elif 'MRI515' in data_dir:
    targ_size = 320
elif 'CT' in data_dir:    
    targ_size = 256

# 遍历每个文件夹
for video_dir in video_dirs:
    print(f"\nProcessing folder: {video_dir}")

    file_names = sorted([f for f in os.listdir(video_dir) if f.endswith(('.jpg', '.png'))])

    for file_name in file_names:
        image_path = os.path.join(video_dir, file_name)

        with Image.open(image_path) as img:
            width, height = img.size
            print(f"  Processing: {file_name}, Original size: {width}x{height}")

            if (width, height) == (targ_size, targ_size):
                print(f"  No resize needed for: {file_name}")
                continue

            ext = os.path.splitext(file_name)[-1].lower()

            if ext == ".jpg":
                # 普通图像使用 LANCZOS
                img_resized = img.resize((targ_size, targ_size), Image.Resampling.LANCZOS)
                img_resized.save(image_path, "JPEG")
                print(f"  Resized JPEG to {targ_size}x{targ_size}")
            
            elif ext == ".png":
                # 标签图：按类别分别 resize，然后合成
                label_np = np.array(img)
                unique_vals = np.unique(label_np)
                combined_mask = np.zeros((targ_size, targ_size), dtype=np.uint8)

                for val in unique_vals:
                    if val == 0:
                        continue  # 背景跳过

                    class_mask = (label_np == val).astype(np.uint8) * 255
                    class_img = Image.fromarray(class_mask, mode='L')
                    resized_class = class_img.resize((targ_size, targ_size), Image.Resampling.NEAREST)
                    resized_class_np = np.array(resized_class)
                    combined_mask[resized_class_np > 127] = val

                Image.fromarray(combined_mask).save(image_path, "PNG")
                print(f"  Resized PNG label with {len(unique_vals)} classes to {targ_size}x{targ_size}")

print("\nAll resizing complete.")



Processing folder: videos/CT_tt/img_in_jpg_L4L5_renamed
  Processing: 00000.jpg, Original size: 299x299
  Resized JPEG to 256x256
  Processing: 00001.jpg, Original size: 283x283
  Resized JPEG to 256x256
  Processing: 00002.jpg, Original size: 308x308
  Resized JPEG to 256x256
  Processing: 00003.jpg, Original size: 263x263
  Resized JPEG to 256x256
  Processing: 00004.jpg, Original size: 277x277
  Resized JPEG to 256x256
  Processing: 00005.jpg, Original size: 287x287
  Resized JPEG to 256x256
  Processing: 00006.jpg, Original size: 296x296
  Resized JPEG to 256x256
  Processing: 00007.jpg, Original size: 305x305
  Resized JPEG to 256x256
  Processing: 00008.jpg, Original size: 272x272
  Resized JPEG to 256x256
  Processing: 00009.jpg, Original size: 264x264
  Resized JPEG to 256x256
  Processing: 00010.jpg, Original size: 288x288
  Resized JPEG to 256x256
  Processing: 00011.jpg, Original size: 279x279
  Resized JPEG to 256x256
  Processing: 00012.jpg, Original size: 285x285
  Resiz

  Resized JPEG to 256x256
  Processing: 00116.jpg, Original size: 331x331
  Resized JPEG to 256x256
  Processing: 00117.jpg, Original size: 313x313
  Resized JPEG to 256x256
  Processing: 00118.jpg, Original size: 279x279
  Resized JPEG to 256x256
  Processing: 00119.jpg, Original size: 333x333
  Resized JPEG to 256x256
  Processing: 00120.jpg, Original size: 275x275
  Resized JPEG to 256x256
  Processing: 00121.jpg, Original size: 297x297
  Resized JPEG to 256x256
  Processing: 00122.jpg, Original size: 287x287
  Resized JPEG to 256x256
  Processing: 00123.jpg, Original size: 307x307
  Resized JPEG to 256x256
  Processing: 00124.jpg, Original size: 261x261
  Resized JPEG to 256x256
  Processing: 00125.jpg, Original size: 285x285
  Resized JPEG to 256x256
  Processing: 00126.jpg, Original size: 317x317
  Resized JPEG to 256x256
  Processing: 00127.jpg, Original size: 333x333
  Resized JPEG to 256x256
  Processing: 00128.jpg, Original size: 267x267
  Resized JPEG to 256x256
  Processing

  Resized JPEG to 256x256
  Processing: 00230.jpg, Original size: 280x280
  Resized JPEG to 256x256
  Processing: 00231.jpg, Original size: 267x267
  Resized JPEG to 256x256
  Processing: 00232.jpg, Original size: 281x281
  Resized JPEG to 256x256
  Processing: 00233.jpg, Original size: 276x276
  Resized JPEG to 256x256
  Processing: 00234.jpg, Original size: 282x282
  Resized JPEG to 256x256
  Processing: 00235.jpg, Original size: 291x291
  Resized JPEG to 256x256
  Processing: 00236.jpg, Original size: 333x333
  Resized JPEG to 256x256
  Processing: 00237.jpg, Original size: 316x316
  Resized JPEG to 256x256
  Processing: 00238.jpg, Original size: 266x266
  Resized JPEG to 256x256
  Processing: 00239.jpg, Original size: 325x325
  Resized JPEG to 256x256
  Processing: 00240.jpg, Original size: 290x290
  Resized JPEG to 256x256
  Processing: 00241.jpg, Original size: 257x257
  Resized JPEG to 256x256
  Processing: 00242.jpg, Original size: 265x265
  Resized JPEG to 256x256
  Processing

  Resized PNG label with 3 classes to 256x256
  Processing: 00092.png, Original size: 285x285
  Resized PNG label with 3 classes to 256x256
  Processing: 00093.png, Original size: 261x261
  Resized PNG label with 3 classes to 256x256
  Processing: 00094.png, Original size: 291x291
  Resized PNG label with 3 classes to 256x256
  Processing: 00095.png, Original size: 333x333
  Resized PNG label with 3 classes to 256x256
  Processing: 00096.png, Original size: 272x272
  Resized PNG label with 3 classes to 256x256
  Processing: 00097.png, Original size: 299x299
  Resized PNG label with 3 classes to 256x256
  Processing: 00098.png, Original size: 295x295
  Resized PNG label with 3 classes to 256x256
  Processing: 00099.png, Original size: 285x285
  Resized PNG label with 3 classes to 256x256
  Processing: 00100.png, Original size: 333x333
  Resized PNG label with 3 classes to 256x256
  Processing: 00101.png, Original size: 267x267
  Resized PNG label with 3 classes to 256x256
  Processing: 

  Resized PNG label with 3 classes to 256x256
  Processing: 00265.png, Original size: 267x267
  Resized PNG label with 3 classes to 256x256
  Processing: 00266.png, Original size: 285x285
  Resized PNG label with 3 classes to 256x256
  Processing: 00267.png, Original size: 315x315
  Resized PNG label with 3 classes to 256x256
  Processing: 00268.png, Original size: 267x267
  Resized PNG label with 3 classes to 256x256
  Processing: 00269.png, Original size: 258x258
  Resized PNG label with 3 classes to 256x256
  Processing: 00270.png, Original size: 321x321
  Resized PNG label with 3 classes to 256x256
  Processing: 00271.png, Original size: 273x430
  Resized PNG label with 3 classes to 256x256
  Processing: 00272.png, Original size: 261x261
  Resized PNG label with 3 classes to 256x256
  Processing: 00273.png, Original size: 298x298
  Resized PNG label with 3 classes to 256x256
  Processing: 00274.png, Original size: 267x267
  Resized PNG label with 3 classes to 256x256
  Processing: 